In [ ]:
import numpy as np
import joblib
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

from sklearn.impute import KNNImputer
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, auc, roc_auc_score, roc_curve

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


from imblearn.over_sampling import SMOTE  # or any other SMOTE variant
from imblearn.combine import SMOTETomek

import shap
from sklearn.inspection import PartialDependenceDisplay




sns.set(style="whitegrid")

## Load Data & Imports

In [ ]:
scd_df = pd.read_csv("/content/drive/MyDrive/Capstone_Trial_Methodology/scd_top18_preprocessed.csv")

# Set up Data Splits

In [ ]:
X = scd_df.drop(columns=['Crisis_Num'])
y = scd_df['Crisis_Num']

In [ ]:
# Split into train, val, and test sets (using stratification for balanced target representation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [ ]:
# Using SMOTETomek resample the X_train, X_val, y_train and y_val
smotetomek = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smotetomek.fit_resample(X_train, y_train)
X_val_resampled, y_val_resampled = smotetomek.fit_resample(X_val, y_val)

In [ ]:
# reassign the resampled sets to the original variables
X_train = X_train_resampled
X_val = X_val_resampled
y_train = y_train_resampled
y_val = y_val_resampled

In [ ]:
# let's clear unused things from memory and prep for training
del X_train_resampled, X_val_resampled, y_train_resampled, y_val_resampled

## A bit of Data Preprocessing


In [ ]:
# update `Pain Location_No Pain` Numbers
X_train[['Pain Location_No Pain', 'Pain Location_Head']] = X_train[['Pain Location_No Pain', 'Pain Location_Head']].astype(int)
X_val[['Pain Location_No Pain', 'Pain Location_Head']] = X_val[['Pain Location_No Pain', 'Pain Location_Head']].astype(int)
X_test[['Pain Location_No Pain', 'Pain Location_Head']] = X_test[['Pain Location_No Pain', 'Pain Location_Head']].astype(int)

X_train

,Pain Intensity,Pain Location_No Pain,Oxygen Saturation (%),Body Temperature (°C),Pain Crises Frequency (past year),Respiratory Rate (bpm),Heart Rate (bpm),Hospitalization History (past year),Acute Chest Syndrome,Dizziness,Pain Location_Head,Nausea,Jaundice,Fever,Shortness of Breath,Swelling,Fatigue,Headache
0,8.000000,0,89.500000,0.0,5.000000,2.000000,2.0,4.000000,4.000000,4.000000,0,2.000000,2.000000,3.000000,0.000000,2.000000,1.000000,5.000000
1,0.000000,1,95.700000,1.0,0.000000,1.000000,1.0,0.000000,1.000000,0.000000,0,0.000000,1.000000,0.000000,3.000000,1.000000,0.000000,1.000000
2,9.000000,0,90.000000,0.0,4.000000,2.000000,1.0,4.000000,2.000000,5.000000,0,4.000000,2.000000,2.000000,3.000000,3.000000,2.000000,4.000000
3,0.000000,1,97.600000,1.0,1.000000,1.000000,1.0,0.000000,0.000000,1.000000,0,0.000000,0.000000,1.000000,1.000000,0.000000,2.000000,2.000000
4,0.000000,1,96.800000,1.0,3.000000,1.000000,1.0,3.000000,0.000000,3.000000,0,3.000000,0.000000,4.000000,4.000000,2.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57533,5.000000,0,95.212867,1.0,2.000000,1.000000,1.0,0.000000,0.467832,1.532168,0,1.467832,2.467832,2.532168,2.532168,3.467832,0.532168,0.532168
57534,5.419260,0,96.037111,1.0,5.000000,1.790370,1.0,3.000000,2.000000,1.209630,0,4.790370,3.209630,4.790370,1.790370,2.000000,4.000000,1.209630
57535,2.775260,0,93.507491,1.0,1.925087,1.074913,1.0,0.000000,0.074913,2.000000,0,4.925087,3.074913,1.925087,1.149827,5.000000,2.000000,2.925087
57536,5.946027,0,95.462219,1.0,1.053973,1.000000,1.0,0.053973,1.000000,2.946027,0,1.053973,3.000000,2.946027,0.053973,0.946027,1.000000,2.946027


# Building an ODE Companion Model for Sickle-Cell Pain-Crisis Dynamics

The goal of this step is to have a mechanistic interpretation + validation of the ML classifiers on the synthetic data we generated from a Gaussian distribution.


## Conceptual Design: 3-Compartment Dynamical System
For this work we regard the population of synthetic patients (or the probability mass of a single patient) as being distributed over three mutually‑exclusive states

| State | Symbol | Clinical Meaning |
| ------ | ------ | ---------------- |
| No Crisis | $C_0(t)$ | baseline, no acute pain |
| Acute Crisis | $C_1(t)$ | vaso occlusive pain episode |
| Chronic Crisis | $C_2(t)$ | presistent pain |


The state vector is now defined as;
$C(t) = [ C_0(t), C_1(t), C_2(t) ]^T$,    $\quad C_0+C_1+C_2=1$

## Transition Structure
We allow:

1. Onset: $C_0 \;\xrightarrow{\;\lambda_{0\to1}\;}\;C_1 \; and \; C_0\xrightarrow{\;\lambda_{0\to2}\;}C_2$

2. Resolution: $C_1\xrightarrow{\;\rho_{1}\;}C_0 , C_2\xrightarrow{\;\rho_{2}\;}C_0$

3. Chronicisation: $C_1\xrightarrow{\;\kappa\;}C_2 (poorly \; resolved \;  acute \;  pain \;  becoming \; chronic)$

## Feature-driven Transition Rates
Each rate is a log-linear hazard driven by the 18 input features $x_j$:

$λ_{0 → 1}(t) = λ_{0 → 1}^{base} exp(∑_{j=1}^{18} β_j^{(0 → 1)} x_j(t))$

Analogous expressions for the other rates $λ_{0 → 2}, \rho_1, \rho_2, κ $

In this study the 18 predictors were left in their raw clinical units; therefore each coefficient $\beta_j$ represents the log-hazard change per one physical unit of that feature (e.g. per 1 bpm increase in heart-rate).

Although a 0-1 rescaling of covariates is common, the log-linear hazard formulation is scale-invariant; we therefore kept features in their original clinical units and interpret each $\beta_j$ as the log-hazard change per unit increase in that measurement.

Interpretation examples:
* If $\beta_{Pain \; Intensity}^{0 → 1} =0.25$ (pain-scale is 0-10), a 4-point rise in pain intensity multiplies the acute-onset hazard by $exp(0.25*4) ≈ 2.7$
* If $\beta_{Heart \; Rate}^{0 → 1} =0.005$, each 10 bpm increase raises the hazard by $exp(0.005*10) = 1.05 ( ≈5\% ) $


Table Guide on Scaling vs Not Scaling

| With 0-1 scaling | With native units |
| ---------------- | ----------------- |
| $β_j$ quantifies the log-hazard change when the feature moves from its minimum to maximum. | $β_j$ quantifies the log-hazard change per one unit of that physical measurement (1 bpm, 1 %, 1 °C, etc.).
| Typical magnitude is $\mid β \mid \tilde \qquad[0.1, 2]$ | Magnitudes may be smaller (e.g $10^{-3}) for large-scale variables or larger for binary flags. |

The coefficients will be from the seeded ML model using values from SHA (e.g SHAP main effect $→$ sign of $β$)

## ODE System
$\frac{dC_0}{dt} = -λ_{0 → 1}C_0  -λ_{0 → 2}C_0 + \rho_1 C_1 + \rho_2 C_2
$

$\frac{dC_1}{dt} = λ_{0 → 1} C_0 - \rho_1C_1 - κC_1$

$\frac{dC_2}{dt} = λ_{0 → 2} C_0 + κC_1 - \rho_2 C_2$

Because the rows sum to zero, $C_0+C_1+C_2 ≡ 1$.


---


*Implementation note*

The ODE is integrated with a differentiable fourth-order Runge-Kutta solver implemented in PyTorch; parameters $θ = \{λ^{base},β\}$ are fitted on GPU using mini-batch Adam (batch = 128, 25 epochs, gradient-norm clipping = 1).
This acceleration is purely computational; it leaves the analytical structure above untouched.


# ODE Implementation

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using", device)

Using cuda


In [ ]:
target_class = 2                          # 0=no crisis, 1=acute, 2=chronic

In [ ]:
# sign shap for all classes
sign_shap_all = {0: {'Pain Intensity': 1,
  'Pain Location_No Pain': -1,
  'Oxygen Saturation (%)': 1,
  'Body Temperature (°C)': 1,
  'Pain Crises Frequency (past year)': -1,
  'Respiratory Rate (bpm)': 1,
  'Heart Rate (bpm)': 1,
  'Acute Chest Syndrome': 1,
  'Hospitalization History (past year)': 1,
  'Dizziness': -1,
  'Pain Location_Head': 1,
  'Nausea': -1,
  'Fever': 1,
  'Jaundice': -1,
  'Shortness of Breath': -1,
  'Swelling': -1,
  'Headache': -1,
  'Fatigue': -1},
 1: {'Pain Intensity': 1,
  'Pain Location_No Pain': 1,
  'Oxygen Saturation (%)': -1,
  'Body Temperature (°C)': -1,
  'Pain Crises Frequency (past year)': 1,
  'Respiratory Rate (bpm)': -1,
  'Heart Rate (bpm)': -1,
  'Acute Chest Syndrome': -1,
  'Hospitalization History (past year)': 1,
  'Dizziness': -1,
  'Pain Location_Head': 1,
  'Nausea': -1,
  'Fever': -1,
  'Jaundice': -1,
  'Shortness of Breath': -1,
  'Swelling': -1,
  'Headache': -1,
  'Fatigue': -1},
 2: {'Pain Intensity': -1,
  'Pain Location_No Pain': 1,
  'Oxygen Saturation (%)': 1,
  'Body Temperature (°C)': 1,
  'Pain Crises Frequency (past year)': -1,
  'Respiratory Rate (bpm)': 1,
  'Heart Rate (bpm)': 1,
  'Acute Chest Syndrome': 1,
  'Hospitalization History (past year)': -1,
  'Dizziness': 1,
  'Pain Location_Head': -1,
  'Nausea': 1,
  'Fever': 1,
  'Jaundice': 1,
  'Shortness of Breath': 1,
  'Swelling': 1,
  'Headache': 1,
  'Fatigue': 1}}

# select sign shap for the target class of interests
sign_shap = sign_shap_all[target_class]
sign_shap

{'Pain Intensity': -1,
 'Pain Location_No Pain': 1,
 'Oxygen Saturation (%)': 1,
 'Body Temperature (°C)': 1,
 'Pain Crises Frequency (past year)': -1,
 'Respiratory Rate (bpm)': 1,
 'Heart Rate (bpm)': 1,
 'Acute Chest Syndrome': 1,
 'Hospitalization History (past year)': -1,
 'Dizziness': 1,
 'Pain Location_Head': -1,
 'Nausea': 1,
 'Fever': 1,
 'Jaundice': 1,
 'Shortness of Breath': 1,
 'Swelling': 1,
 'Headache': 1,
 'Fatigue': 1}

## Dataset --> GPU Tensors

In [ ]:
# X_scaled, y_labels are NumPy arrays from your pre‑processing
X_t = torch.tensor(X_train.values, dtype=torch.float32, device=device)
y_t = torch.tensor(y_train.values, dtype=torch.long,   device=device)

ds = TensorDataset(X_t, y_t)
loader = DataLoader(ds, batch_size=128, shuffle=True)

## ODE in PyTorch

In [ ]:
N_FEAT = 18
STATE0 = torch.tensor([1., 0., 0.], device=device)   # initial [C0,C1,C2]

class ODEModel(nn.Module):
    def __init__(self, sign_vector):
        super().__init__()
        # log‑base rates  (5 parameters)
        self.log_base = nn.ParameterDict({
            k: nn.Parameter(torch.log(torch.tensor(v, device=device)))
            for k, v in {
                'lam01': 0.02, 'lam02': 0.005,
                'rho1':  0.03, 'rho2':  0.01,
                'kap':   0.002
            }.items()
        })
        # β matrices  (5×18)
        for k in self.log_base:
            init = 0.01 * torch.tensor(sign_vector, device=device, dtype=torch.float32)
            self.register_parameter(f"beta_{k}",
                                    nn.Parameter(init.clone()))

    # -----------------------------------------------
    # RK4 loop
    def forward(self, x, horizon_h):
        dt = 1.0 / 24
        steps = int(horizon_h)
        C = STATE0.repeat(x.size(0), 1)

        for _ in range(steps):
            k1 = self._rhs(C, x)
            k2 = self._rhs(C + 0.5*dt*k1, x)
            k3 = self._rhs(C + 0.5*dt*k2, x)
            k4 = self._rhs(C + dt*k3, x)
            C = C + dt/6 * (k1 + 2*k2 + 2*k3 + k4)

            # ------- renormalise & clamp --------------------
            C = torch.clamp(C, 1e-6, 1.0)
            C = C / C.sum(dim=1, keepdim=True)

        return C

    # -----------------------------------------------
    def _rhs(self, C, x):
        rates = {}
        for k in self.log_base:
            beta = getattr(self, f"beta_{k}")                    # (18,)
            # ----- log‑rate (clip to ±10) --------------------
            log_rate = self.log_base[k] + (x * beta).sum(dim=1)
            log_rate = torch.clamp(log_rate, -10.0, 10.0)
            rates[k] = torch.exp(log_rate)                       # (B,)

        C = torch.clamp(C, 0.0, 1.0)                             # avoid drift <0
        C0, C1, C2 = C[:, 0], C[:, 1], C[:, 2]
        dC0 = -(rates['lam01']+rates['lam02'])*C0 + rates['rho1']*C1 + rates['rho2']*C2
        dC1 =  rates['lam01']*C0 - (rates['rho1']+rates['kap'])*C1
        dC2 =  rates['lam02']*C0 + rates['kap']*C1 - rates['rho2']*C2
        return torch.stack([dC0, dC1, dC2], dim=1)

## Training loop (mini‑batch Adam)

In [ ]:
# SHAP sign vector (+1/0/‑1) for chronic crisis, length 18
FEATURES = [
    "Pain Intensity", "Pain Location_No Pain", "Oxygen Saturation (%)",
    "Body Temperature (°C)", "Pain Crises Frequency (past year)",
    "Respiratory Rate (bpm)", "Heart Rate (bpm)", "Acute Chest Syndrome",
    "Hospitalization History (past year)", "Dizziness", "Pain Location_Head",
    "Nausea", "Fever", "Jaundice", "Shortness of Breath",
    "Swelling", "Headache", "Fatigue"
]

sign_vector = [sign_shap[f] for f in FEATURES]        # not dict order!
sign_vec    = torch.tensor(sign_vector,
                           dtype=torch.float32,
                           device=device)

sign_vec

tensor([-1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.], device='cuda:0')

In [ ]:
model = ODEModel(sign_vec).to(device)
opt = torch.optim.Adam(model.parameters(), lr=5e-3)   # half the LR
criterion = nn.NLLLoss()          # we’ll use log‑probabilities
clip_value = 1.0

HORIZON = 24    # fit to 24‑h labels (same as ensemble)

for epoch in range(30):
    total_loss = 0.
    for xb, yb in loader:
        opt.zero_grad()
        probs = model(xb, HORIZON)

        logp  = torch.log(probs)
        loss  = criterion(logp, yb)

        if torch.isnan(loss):
            raise RuntimeError("NaN loss encountered")

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        opt.step()
        total_loss += loss.item()*xb.size(0)

    print(f"Epoch {epoch:02d}  NLL={total_loss/len(ds):.4f}")

<ipython-input-110-d005cff51534>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  init = 0.01 * torch.tensor(sign_vector, device=device, dtype=torch.float32)


Epoch 00  NLL=0.2369
Epoch 01  NLL=0.1270
Epoch 02  NLL=0.0982
Epoch 03  NLL=0.5107
Epoch 04  NLL=0.3086
Epoch 05  NLL=0.9937
Epoch 06  NLL=0.9626
Epoch 07  NLL=0.4444
Epoch 08  NLL=0.8756
Epoch 09  NLL=1.1007
Epoch 10  NLL=0.2612
Epoch 11  NLL=0.2846
Epoch 12  NLL=0.8947
Epoch 13  NLL=0.7979
Epoch 14  NLL=0.7191
Epoch 15  NLL=1.1571
Epoch 16  NLL=0.3699
Epoch 17  NLL=0.2955
Epoch 18  NLL=0.6936
Epoch 19  NLL=0.2309
Epoch 20  NLL=0.1082
Epoch 21  NLL=0.1372
Epoch 22  NLL=0.1097
Epoch 23  NLL=0.1054
Epoch 24  NLL=0.1075
Epoch 25  NLL=0.0915
Epoch 26  NLL=0.0929
Epoch 27  NLL=0.0923
Epoch 28  NLL=0.1218
Epoch 29  NLL=0.0889


In [ ]:
# save model
torch.save(model.state_dict(), "/content/drive/MyDrive/Capstone_Trial_Methodology/models/ode_theta_torch.pt")

## Evaluations

In [ ]:
from torch.utils.data import DataLoader, TensorDataset


@torch.no_grad()
def torch_ode_predict_proba(model, X_scaled, horizon_h=24, batch=512):
    """
    model      : trained ODEModel (in .eval() mode)
    X_scaled   : NumPy array (N,18) already scaled
    horizon_h  : forecast horizon in hours
    batch      : mini‑batch size
    returns    : NumPy array  (N,3)
    """
    model.eval()
    ds = TensorDataset(torch.tensor(X_scaled, dtype=torch.float32, device=device))
    dl = DataLoader(ds, batch_size=batch, shuffle=False)

    out_list = []
    for (xb,) in dl:
        probs = model(xb, horizon_h)           # (B,3)
        out_list.append(probs.cpu())           # back to CPU tensor
    P = torch.cat(out_list, dim=0).numpy()
    return P

### Evaluation on validation set

In [ ]:
print("****** Evaluating of ODE on Validation Set ******")

HORIZON = 24          # same horizon you fitted on
P_val   = torch_ode_predict_proba(model, X_val.values, horizon_h=HORIZON)

y_pred  = np.argmax(P_val, axis=1)

print(f"PyTorch‑ODE accuracy     : {accuracy_score(y_test, y_pred):.4f} ")
print(f"PyTorch‑ODE f1‑macro     : {f1_score(y_test, y_pred, average='weighted') :.4f}")
print(f"PyTorch‑ODE ROC‑AUC (OvR): {roc_auc_score(label_binarize(y_test, classes=[0,1,2]), P_val, multi_class='ovr', average='weighted'):.4f}")

****** Evaluating on Validation Set ******
PyTorch‑ODE accuracy     : 0.9623529411764706
PyTorch‑ODE f1‑macro     : 0.9623366585628764
PyTorch‑ODE ROC‑AUC (OvR): 0.995774953054732


### Evaluation on test set

In [ ]:
print("****** Evaluating of ODE on Test Set ******")

HORIZON = 24          # same horizon you fitted on
P_val   = torch_ode_predict_proba(model, X_test.values, horizon_h=HORIZON)

y_pred  = np.argmax(P_val, axis=1)

print(f"PyTorch‑ODE accuracy     : {accuracy_score(y_test, y_pred):.4f} ")
print(f"PyTorch‑ODE f1‑weighted     : {f1_score(y_test, y_pred, average='weighted') :.4f}")
print(f"PyTorch‑ODE ROC‑AUC (OvR): {roc_auc_score(label_binarize(y_test, classes=[0,1,2]), P_val, multi_class='ovr', average='weighted'):.4f}")

****** Evaluating of ODE on Test Set ******
PyTorch‑ODE accuracy     : 0.9641 
PyTorch‑ODE f1‑macro     : 0.9648
PyTorch‑ODE ROC‑AUC (OvR): 0.9978


# Extract coefficients from the fitted model

In [ ]:
model_path = "/content/drive/MyDrive/Capstone_Trial_Methodology/models/ode_theta_torch.pt"

ode_model = ODEModel(sign_vector).to(device)
ode_model.load_state_dict(torch.load(model_path, map_location=device))
ode_model.eval()

ODEModel(
  (log_base): ParameterDict(
      (kap): Parameter containing: [torch.cuda.FloatTensor of size  (cuda:0)]
      (lam01): Parameter containing: [torch.cuda.FloatTensor of size  (cuda:0)]
      (lam02): Parameter containing: [torch.cuda.FloatTensor of size  (cuda:0)]
      (rho1): Parameter containing: [torch.cuda.FloatTensor of size  (cuda:0)]
      (rho2): Parameter containing: [torch.cuda.FloatTensor of size  (cuda:0)]
  )
)

In [ ]:
# -------------------------------------------------
# assume `ode_model` is the trained model already on CPU or GPU
# -------------------------------------------------
transitions = {
    "lam01": "C0 → C1  (acute onset)",
    "lam02": "C0 → C2  (chronic onset)",
    "rho1" : "C1 → C0  (acute resolution)",
    "rho2" : "C2 → C0  (chronic resolution)",
    "kap"  : "C1 → C2  (chronicisation)"
}

rows = []
for key, label in transitions.items():
    beta_vec = getattr(ode_model, f"beta_{key}").detach().cpu().numpy()
    for feat, beta in zip(FEATURES, beta_vec):
        rows.append({"transition": key,
                     "description": label,
                     "feature": feat,
                     "beta": float(beta)})

beta_df = pd.DataFrame(rows)

# Base (log) rates and their exponentiated values
base_rows = [
    {"transition": k,
     "description": transitions[k],
     "log_rate": float(ode_model.log_base[k].detach().cpu()),
     "base_rate_per_hour": float(torch.exp(ode_model.log_base[k]-torch.log(torch.tensor(24.0))))}
    for k in transitions
]
base_df = pd.DataFrame(base_rows)

In [ ]:
# write beta-coefficients to a file
beta_df.to_csv("/content/drive/MyDrive/Capstone_Trial_Methodology/beta_coefficients.csv", index=False)

print(" β-coefficients (first 5 rows) ")
beta_df.head()

 β-coefficients (first 5 rows) 


,transition,description,feature,beta
0,lam01,C0 → C1 (acute onset),Pain Intensity,0.369561
1,lam01,C0 → C1 (acute onset),Pain Location_No Pain,-5.754713
2,lam01,C0 → C1 (acute onset),Oxygen Saturation (%),-0.081097
3,lam01,C0 → C1 (acute onset),Body Temperature (°C),-1.058929
4,lam01,C0 → C1 (acute onset),Pain Crises Frequency (past year),0.339946


In [ ]:
print(" Base hazard rates ")
base_df

 Base hazard rates 


,transition,description,log_rate,base_rate_per_hour
0,lam01,C0 → C1 (acute onset),-2.287006,0.004232
1,lam02,C0 → C2 (chronic onset),-6.720471,0.000050
2,rho1,C1 → C0 (acute resolution),-3.679070,0.001052
3,rho2,C2 → C0 (chronic resolution),-4.570116,0.000432
4,kap,C1 → C2 (chronicisation),-6.278670,0.000078
